## Overview

- In this notebook, I am compiling multiple functions to create a final function that will calculate demographic proportions for all of our defined neighborhoods in Chicago within a specific distance (20 minutes walking) of each library.

- This notebook breaks down into:
    - Importing Packages & Loading Datasets
    - Copying Coverage Function (made by Abby/Rajvi)
    - 6 Individual Demographic Functions 
    - All Branch Demographics (combining 6 functions into 1)
    - Final Function That Merges Library Neighborhoods with Demographics

## Importing Packages & Loading Datasets

**Importing Packages**

In [72]:
#importing necessary libraries
from rajvi_mapbox_api import MapboxAPI
import requests
from urllib.request import urlopen
import json
from shapely.geometry import shape, Polygon
import numpy as np
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt
import seaborn as sns
plt.style.use('default')
import geopandas as gpd
import geodatasets
import ast
from shapely import wkt
from pyproj import Geod

**Loading in Datasets**

In [86]:
#Has geometries for our defined Neighborhoods
branchInfo = pd.read_csv('../library neighborhoods/completedDrivingNeighborhoodRadiiDF.csv')
#Population Demographics for below function
censusTracts = pd.read_csv('../data/clean/population_demographics.csv')
#Census Demographics for each census tract
censusDemographics = pd.read_csv('../data/dani data/census_demos.csv')
avgDriving = pd.read_csv('../library neighborhoods/averageDrivingTimeNeighborhoods.csv')

**Removing Unneccesary Columns from Datasets**

In [87]:
#removing unecessary columns from branchInfo
branchInfo = branchInfo.drop(columns = ['Unnamed: 0'])
#removing unecessary columns from censusTracts
censusTracts = censusTracts[['geoid','geometry','qualifying name']]

## Copying Coverage Function ( from Rajvi's code for neighborhoods)

In [88]:
#converting censusTracts to a geodataframe (we could not load it as one since it already had a geometry column so we are simply converting the geometry column to a proper gpd geometry column).
censusTracts = gpd.GeoDataFrame(
    censusTracts.loc[:, [c for c in censusTracts.columns if c != "geometry"]],
    geometry=gpd.GeoSeries.from_wkt(censusTracts["geometry"]),
    crs="epsg:4326",
    )

In [89]:
branchInfo[['LATITUDE', 'LONGITUDE']] = [ast.literal_eval(x)[:2] for x in branchInfo['LOCATION']]
branchInfo.loc[:, 'LATITUDE'] = pd.to_numeric(branchInfo.loc[:, 'LATITUDE'])
branchInfo.loc[:, 'LONGITUDE'] = pd.to_numeric(branchInfo.loc[:, 'LONGITUDE'])

In [90]:
branchInfo = branchInfo.drop(columns = ['LOCATION', 'LATITUDE', 'LONGITUDE'])

In [91]:
branchInfo = branchInfo.reset_index()

In [92]:
branchInfo = branchInfo.drop(columns = ['index'])

In [93]:
branchInfo = gpd.GeoDataFrame(
    branchInfo.loc[:, [c for c in branchInfo.columns if c != "geometry"]],
    geometry=gpd.GeoSeries.from_wkt(branchInfo["geometry"]),
    crs="epsg:4326",
    )
branchInfo.head(3)

,BRANCH,geometry
0,Albany Park,"MULTIPOLYGON (((-87.72721 41.97358, -87.72759 ..."
1,Altgeld,"MULTIPOLYGON (((-87.59243 41.66233, -87.59242 ..."
2,Archer Heights,"MULTIPOLYGON (((-87.73425 41.79026, -87.73574 ..."


In [94]:
#helper method to ensure that two polygons overlap before getting the intersection. Gets the intersection then calculates
#the percent of overlapping area for a census tract and saves it to a dictionary with the census tract geo-id as the key and
#the percent as the value.
def check(polygon1, df, geoname, dictionary,key):
   
    for i in range(len(df)):
        if polygon1.intersects(df.loc[i,geoname]): 
            overlapPolygon = (polygon1.intersection(df.loc[i,geoname]))
            poly_area, poly_perimeter = geod.geometry_area_perimeter(overlapPolygon)
            overlapArea = poly_area*-1
            propOverlap = overlapArea / df.loc[i,'Area']
            #print(propOverlap)
            dictionary[df.loc[i, key]] = propOverlap
    return dictionary
   

In [95]:
#sets the crs of gpd types
def geoSetup(gdf):
    
    gdf = gdf.set_crs('EPSG:4326')
    

In [96]:
#cdf = gdf of geometry area which you want to get the coverage score
#idf = gdf of points for which you want to get an isochrone and dictionary of percent of overlap in cdf areas
#scoreString = a string that is the name of the coverage score column you choose
#dictString = a string that is the name of the dictionary column you choose
#cgs = a string that is the name of the geometry column in the cdf
#igs = a string that is the name of the geometry column in the idf
#lat = a string that is the name of the latitude column in the idf
#lon = a string that is the name of the longitude column in the idf
#key = a string that is the name of the column in the cdf that you want to represent the key of the dictionary

def appendADS(cdf,idf,scoreString,dictString,cgs,igs,key):    
    #api = MapboxAPI()
    cdf[scoreString] = 0.0
    idf[dictString] = ''
    censusAreas = []
    global geod 
    geod = Geod(ellps='WGS84')
   
    geoSetup(cdf)
    geoSetup(idf)
   
    for index, row in cdf.iterrows():
        poly_area, poly_perimeter = geod.geometry_area_perimeter(row[cgs])
        poly_area = poly_area*-1
        censusAreas.append(poly_area)
    
    cdf['Area'] = censusAreas



    for i in range(len(idf)):
        dictionary = {}
        dictionary = check(idf.loc[i,igs],cdf,cgs, dictionary,key)
        branchInfo.loc[i,dictString]= [dictionary]
    
        for i in range(len(cdf)):
   
            if dictionary.get(cdf.loc[i, key]) != None:
                score = cdf.loc[i, scoreString]
                cdf.loc[i, scoreString] = score + dictionary.get(cdf.loc[i, key])

In [97]:
appendADS(censusTracts,branchInfo, 'score', 'list of dict','geometry','geometry','geoid')

GEOSException: AssertionFailedException: found two shells in EdgeRing list

In [85]:
branchInfo.tail(3)

,BRANCH,geometry,list of dict
78,"Whitney M. Young, Jr.","MULTIPOLYGON (((-87.60798 41.75899, -87.60760 ...","[{17031520100: 3.286179546983481e-05, 17031490..."
79,Woodson Regional Library,"MULTIPOLYGON (((-87.65605 41.72356, -87.65405 ...","[{17031720700: 0.014170836361329823, 170317202..."
80,Wrightwood-Ashburn,"MULTIPOLYGON (((-87.70991 41.74565, -87.70810 ...","[{17031720100: 0.013844478902345757, 170317112..."


## 6 Individual Demographic Functions

**These 6 functions iterate through every possible demographic variable from the census_demos.csv for each overlapping census tract, and calculate one normalized value for every variable.**

In [18]:
#Internet
def internet_by_branch(DemographicsVariables, branchWithCensus):
    ''' We are creating a function that allows me to retrieve only the internet census data, and then compare this with Altgeld'''
    pop_in_branch = (branchWithCensus['Percent Overlap']  * branchWithCensus['Total']).sum()
    Branch_pop_withInternet = (branchWithCensus['Percent Overlap'] * branchWithCensus[DemographicsVariables] * branchWithCensus['Total']).sum() 
    result = Branch_pop_withInternet/pop_in_branch
    return result   

#Income
def income_by_branch(DemographicsVariables, branchWithCensus):
    if DemographicsVariables == 'Percent Population in Labor Force 16+: Employed' or DemographicsVariables == 'Percent Population in Labor Force 16+: Unemployed':
        print(DemographicsVariables)
        pop_in_branch = (branchWithCensus['Percent Overlap']  * branchWithCensus['civilian population in labor force 16 years and over']).sum()
        Branch_pop_withIncome = (branchWithCensus['Percent Overlap'] * branchWithCensus[DemographicsVariables] * branchWithCensus['civilian population in labor force 16 years and over']).sum() 
        result = Branch_pop_withIncome/pop_in_branch
        return result
    elif DemographicsVariables == 'Percent Households: Less Than $10,000' or DemographicsVariables == 'Percent Households: $10,000 to $14,999' or DemographicsVariables == 'Percent Households: $15,000 to $19,999' or DemographicsVariables == 'Percent Households: $20,000 to $24,999' or DemographicsVariables == 'Percent Households: $25,000 to $29,999' or DemographicsVariables == 'Percent Households: $30,000 to $34,999' or DemographicsVariables == 'Percent Households: $35,000 to $39,999' or DemographicsVariables == 'Percent Households: $40,000 to $44,999' or DemographicsVariables == 'Percent Households: $45,000 to $49,999' or DemographicsVariables == 'Percent Households: $50,000 to $59,999' or DemographicsVariables == 'Percent Households: $60,000 to $74,999' or DemographicsVariables == 'Percent Households: $75,000 to $99,999' or DemographicsVariables == 'Percent Households: $100,000 to $124,999' or DemographicsVariables == 'Percent Households: $125,000 to $149,999' or DemographicsVariables == 'Percent Households: $150,000 to $199,999' or DemographicsVariables == 'Percent Households: $200,000 or more':
        print(DemographicsVariables)
        pop_in_branch = (branchWithCensus['Percent Overlap']  * branchWithCensus['total households reporting income']).sum()
        Branch_pop_withIncome = (branchWithCensus['Percent Overlap'] * branchWithCensus[DemographicsVariables] * branchWithCensus['total households reporting income']).sum() 
        result = Branch_pop_withIncome/pop_in_branch
        return result
    elif DemographicsVariables == 'Percent Ages 18-64: Living in Poverty' or DemographicsVariables == 'Percent Ages 18-64: At or Above Poverty Level':
        print(DemographicsVariables)
        pop_in_branch = (branchWithCensus['Percent Overlap']  * branchWithCensus['population age 18 to 64 for whom poverty status  is determined']).sum()
        Branch_pop_withIncome = (branchWithCensus['Percent Overlap'] * branchWithCensus[DemographicsVariables] * branchWithCensus['population age 18 to 64 for whom poverty status  is determined']).sum() 
        result = Branch_pop_withIncome/pop_in_branch
        return result
    elif DemographicsVariables == 'Percent Households: With Public Assistance Income' or DemographicsVariables == 'Percent Households: No Public Assistance Income':
        print(DemographicsVariables)
        pop_in_branch = (branchWithCensus['Percent Overlap']  * branchWithCensus['total households reporting public assistance income status']).sum()
        Branch_pop_withIncome = (branchWithCensus['Percent Overlap'] * branchWithCensus[DemographicsVariables] * branchWithCensus['total households reporting public assistance income status']).sum() 
        result = Branch_pop_withIncome/pop_in_branch
        return result

# #Commute
def commute_by_branch(DemographicsVariables, branchWithCensus):
    if DemographicsVariables == 'Percent Owner and Renter Occupied Housing: No Vehicles' or DemographicsVariables == 'Percent Owner and Renter Occupied Housing: 1 Vehicle' or DemographicsVariables == 'Percent Owner and Renter Occupied Housing: 2 Vehicles' or DemographicsVariables == 'Percent Owner and Renter Occupied Housing: 3 Vehicles' or DemographicsVariables == 'Percent Owner and Renter Occupied Housing: 4 Vehicles' or DemographicsVariables == 'Percent Owner and Renter Occupied Housing: 5 or More Vehicles':
        print(DemographicsVariables)
        pop_in_branch = (branchWithCensus['Percent Overlap']  * branchWithCensus['occupied housing units']).sum()
        Branch_pop_withCommute = (branchWithCensus['Percent Overlap'] * branchWithCensus[DemographicsVariables] * branchWithCensus['occupied housing units']).sum() 
        result = Branch_pop_withCommute/pop_in_branch
        return result
    elif DemographicsVariables == 'Percent Renter-Occupied Housing: No Vehicles' or DemographicsVariables == 'Percent Renter-Occupied Housing: 1 Vehicle' or DemographicsVariables == 'Percent Renter-Occupied Housing: 2 Vehicles' or DemographicsVariables == 'Percent Renter-Occupied Housing: 3 Vehicles' or DemographicsVariables == 'Percent Renter-Occupied Housing: 4 Vehicles' or DemographicsVariables == 'Percent Renter-Occupied Housing: 5 or More Vehicles':
        print(DemographicsVariables)
        pop_in_branch = (branchWithCensus['Percent Overlap']  * branchWithCensus['renter-occupied housing units']).sum()
        Branch_pop_withCommute = (branchWithCensus['Percent Overlap'] * branchWithCensus[DemographicsVariables] * branchWithCensus['renter-occupied housing units']).sum() 
        result = Branch_pop_withCommute/pop_in_branch
        return result
    elif DemographicsVariables == 'Percent Workers 16+: Car, Truck, or Van ' or DemographicsVariables == 'Percent Workers 16+: Drove Alone' or DemographicsVariables == 'Percent Workers 16+: Carpooled' or DemographicsVariables == 'Percent Workers 16+: Public Transportation (Includes Taxi)' or DemographicsVariables == 'Percent Workers 16+: Motorcycle' or DemographicsVariables == 'Percent Workers 16+: Bicycle' or DemographicsVariables == 'Percent Workers 16+: Walked' or DemographicsVariables == 'Percent Workers 16+: Other Means':
        print(DemographicsVariables)
        pop_in_branch = (branchWithCensus['Percent Overlap']  * branchWithCensus['workers 16 years and over']).sum()
        Branch_pop_withCommute = (branchWithCensus['Percent Overlap'] * branchWithCensus[DemographicsVariables] * branchWithCensus['workers 16 years and over']).sum() 
        result = Branch_pop_withCommute/pop_in_branch
        return result

#Race
def race_by_branch(DemographicsVariables, branchWithCensus):
    """ This function takes the population of each variables and devide it by the population to give us the perecntage of how many people have a specific Race in the overlapping area of the branch"""
    print(DemographicsVariables)
    pop_in_branch = (branchWithCensus['Percent Overlap']  * branchWithCensus['total population']).sum()
    Branch_pop_withRace = (branchWithCensus['Percent Overlap'] * branchWithCensus[DemographicsVariables] * branchWithCensus['total population']).sum()
    result = Branch_pop_withRace/pop_in_branch
    return result

#Age
def age_by_branch(DemographicsVariables, branchWithCensus):
    """ This function takes the population of each variables and devide it by the population to give us the perecntage of how many people have a specific age in the overlapping area of the branch"""
    print(DemographicsVariables)
    pop_in_branch = (branchWithCensus['Percent Overlap']  * branchWithCensus['total population']).sum()
    Branch_pop_withAge = (branchWithCensus['Percent Overlap'] * branchWithCensus[DemographicsVariables] * branchWithCensus['total population']).sum()
    result = Branch_pop_withAge/pop_in_branch
    return result

#Education
def education_by_branch(DemographicsVariables, branchWithCensus):
    """ This function takes the population of each variables and devide it by the population to give us the perecntage of how many people have a specific degree in the overlapping area of the branch"""
    print(DemographicsVariables)
    pop_in_branch = (branchWithCensus['Percent Overlap']  * branchWithCensus['population 25 years and over']).sum()
    Branch_pop_withEducation = (branchWithCensus['Percent Overlap'] * branchWithCensus[DemographicsVariables] * branchWithCensus['population 25 years and over']).sum()
    result = Branch_pop_withEducation/pop_in_branch
    return result


- So, for these functions, I'd have to specifically call the name of one of the branches, and the specific variable I want (ex: education_by_branch('Percent: 25+ High School or More')) to return the value I need.
- But, I want to run through all of those possible scenarios for each function. I then want to see all of these values in a DataFrame. This requires me to create a function that will first take in the above functions that iterate through all of the demographic variables. Then, the function must also contain another function that will run through all of the library neighborhoods for every variable. **The below function does this.**

## All Branch Demographics (combining 6 functions into 1)

In [19]:
def branch_demographics(branchWithCensus, nameOfBranch):
    branch_snapshot_df = pd.DataFrame({'Branch': [nameOfBranch]})
    list1 = ['Percent of People with Internet Access',
             'Percent of People without Internet Access',
             'Percent of People with Computers', 
             'Percent of People without Computers', 
             'Percent of People with Computers and Internet']
    for i in list1:
        branch_snapshot_df[i] = internet_by_branch(i, branchWithCensus)
    
    list2 = ['Percent Population in Labor Force 16+: Employed',
             'Percent Population in Labor Force 16+: Unemployed',
             'Percent Households: Less Than $10,000',
             'Percent Households: $10,000 to $14,999',
             'Percent Households: $15,000 to $19,999',
             'Percent Households: $20,000 to $24,999',
             'Percent Households: $25,000 to $29,999',
             'Percent Households: $30,000 to $34,999',
             'Percent Households: $35,000 to $39,999',
             'Percent Households: $40,000 to $44,999',
             'Percent Households: $45,000 to $49,999',
             'Percent Households: $50,000 to $59,999',
             'Percent Households: $60,000 to $74,999',
             'Percent Households: $75,000 to $99,999',
             'Percent Households: $100,000 to $124,999',
             'Percent Households: $125,000 to $149,999',
             'Percent Households: $150,000 to $199,999',
             'Percent Households: $200,000 or more',
             'Percent Ages 18-64: Living in Poverty',
             'Percent Ages 18-64: At or Above Poverty Level',
             'Percent Households: With Public Assistance Income',
             'Percent Households: No Public Assistance Income']
    for i in list2:
        branch_snapshot_df[i] = income_by_branch(i, branchWithCensus)
    
    list3 = ['Percent Owner and Renter Occupied Housing: No Vehicles',
             'Percent Owner and Renter Occupied Housing: 1 Vehicle',
             'Percent Owner and Renter Occupied Housing: 2 Vehicles',
             'Percent Owner and Renter Occupied Housing: 3 Vehicles',
             'Percent Owner and Renter Occupied Housing: 4 Vehicles',
             'Percent Owner and Renter Occupied Housing: 5 or More Vehicles',
             'Percent Renter-Occupied Housing: No Vehicles',
             'Percent Renter-Occupied Housing: 1 Vehicle',
             'Percent Renter-Occupied Housing: 2 Vehicles',
             'Percent Renter-Occupied Housing: 3 Vehicles',
             'Percent Renter-Occupied Housing: 4 Vehicles',
             'Percent Renter-Occupied Housing: 5 or More Vehicles',
             'Percent Workers 16+: Car, Truck, or Van ',
             'Percent Workers 16+: Drove Alone',
             'Percent Workers 16+: Carpooled',
             'Percent Workers 16+: Public Transportation (Includes Taxi)',
             'Percent Workers 16+: Motorcycle',
             'Percent Workers 16+: Bicycle',
             'Percent Workers 16+: Walked',
             'Percent Workers 16+: Other Means']
    for i in list3:
        branch_snapshot_df[i] = commute_by_branch(i, branchWithCensus)
    
    list4 = ['Percent: White Alone',
             'Percent: Black or African American Alone',
             'Percent: American Indian and Alaska Native Alone',
             'Percent: Asian Alone',
             'Percent: Native Hawaiian and Other Pacific Islander Alone',
             'Percent: Some Other Race Alone',
             'Percent: Two or More Races']
    for i in list4:
        branch_snapshot_df[i] = race_by_branch(i, branchWithCensus)
        
    list5 = ['Percent: under 5 years',
             'Percent: 5 to 9 years',
             'Percent: 10 to 14 years',
             'Percent: 15 to 19 years',
             'Percent: 20 to 24 years',
             'Percent: 25 to 29 years',
             'Percent: 30 to 34 years',
             'Percent: 35 to 39 years',
             'Percent: 50 to 54 years',
             'Percent: 55 to 59 years',
             'Percent: 60 to 64 years',
             'Percent: 65 to 69 years',
             'Percent: 70 to 74 years',
             'Percent: 75 to 79 years',
             'Percent: 80 to 84 years',
             'Percent: 85 years and over']
    for i in list5:
        branch_snapshot_df[i] = age_by_branch(i, branchWithCensus)
    
    list6 = ['Percent: 25+ Less Than High School', 
             'Percent: 25+ High School or More', 
             'Percent: 25+ Some College or More', 
             'Percent: 25+ Bachelor\'s Degree or More', 
             'Percent: 25+ Master\'s Degree or More',
             'Percent: 25+ Professional School Degree or More',
             'Percent: 25+ Doctorate Degree']
    for i in list6: 
        branch_snapshot_df[i] = education_by_branch(i, branchWithCensus)
       
    return branch_snapshot_df

## Final Function That Merges Library Neighborhoods with Demographics

**- The previous functions were intially made to only work on one library. The 'for' loop inside this function allows me to get values for all of the library branches.**

In [20]:
def allBranchDemographics(branchDataset):
    dataByBranch = pd.DataFrame(columns=['Branch',
                                        'Percent of People with Internet Access',
                                        'Percent of People without Internet Access',
                                        'Percent of People with Computers',
                                        'Percent of People without Computers',
                                        'Percent of People with Computers and Internet',
                                        'Percent Population in Labor Force 16+: Employed',
                                        'Percent Population in Labor Force 16+: Unemployed',
                                        'Percent Households: Less Than $10,000',
                                        'Percent Households: $10,000 to $14,999',
                                        'Percent Households: $15,000 to $19,999',
                                        'Percent Households: $20,000 to $24,999',
                                        'Percent Households: $25,000 to $29,999',
                                        'Percent Households: $30,000 to $34,999',
                                        'Percent Households: $35,000 to $39,999',
                                        'Percent Households: $40,000 to $44,999',
                                        'Percent Households: $45,000 to $49,999',
                                        'Percent Households: $50,000 to $59,999',
                                        'Percent Households: $60,000 to $74,999',
                                        'Percent Households: $75,000 to $99,999',
                                        'Percent Households: $100,000 to $124,999',
                                        'Percent Households: $125,000 to $149,999',
                                        'Percent Households: $150,000 to $199,999',
                                        'Percent Households: $200,000 or more',
                                        'Percent Ages 18-64: Living in Poverty',
                                        'Percent Ages 18-64: At or Above Poverty Level',
                                        'Percent Households: With Public Assistance Income',
                                        'Percent Households: No Public Assistance Income',
                                        'Percent Owner and Renter Occupied Housing: No Vehicles',
                                        'Percent Owner and Renter Occupied Housing: 1 Vehicle',
                                        'Percent Owner and Renter Occupied Housing: 2 Vehicles',
                                        'Percent Owner and Renter Occupied Housing: 3 Vehicles',
                                        'Percent Owner and Renter Occupied Housing: 4 Vehicles',
                                        'Percent Owner and Renter Occupied Housing: 5 or More Vehicles',
                                        'Percent Renter-Occupied Housing: No Vehicles',
                                        'Percent Renter-Occupied Housing: 1 Vehicle',
                                        'Percent Renter-Occupied Housing: 2 Vehicles',
                                        'Percent Renter-Occupied Housing: 3 Vehicles',
                                        'Percent Renter-Occupied Housing: 4 Vehicles',
                                        'Percent Renter-Occupied Housing: 5 or More Vehicles',
                                        'Percent Workers 16+: Car, Truck, or Van ',
                                        'Percent Workers 16+: Drove Alone',
                                        'Percent Workers 16+: Carpooled',
                                        'Percent Workers 16+: Public Transportation (Includes Taxi)',
                                        'Percent Workers 16+: Motorcycle',
                                        'Percent Workers 16+: Bicycle',
                                        'Percent Workers 16+: Walked',
                                        'Percent Workers 16+: Other Means',
                                        'Percent: White Alone',
                                        'Percent: Black or African American Alone',
                                        'Percent: American Indian and Alaska Native Alone',
                                        'Percent: Asian Alone',
                                        'Percent: Native Hawaiian and Other Pacific Islander Alone',
                                        'Percent: Some Other Race Alone',
                                        'Percent: Two or More Races',
                                        'Percent: under 5 years',
                                        'Percent: 5 to 9 years',
                                        'Percent: 10 to 14 years',
                                        'Percent: 15 to 19 years',
                                        'Percent: 20 to 24 years',
                                        'Percent: 25 to 29 years',
                                        'Percent: 30 to 34 years',
                                        'Percent: 35 to 39 years',
                                        'Percent: 50 to 54 years',
                                        'Percent: 55 to 59 years',
                                        'Percent: 60 to 64 years',
                                        'Percent: 65 to 69 years',
                                        'Percent: 70 to 74 years',
                                        'Percent: 75 to 79 years',
                                        'Percent: 80 to 84 years',
                                        'Percent: 85 years and over',
                                        'Percent: 25+ Less Than High School', 
                                        'Percent: 25+ High School or More', 
                                        'Percent: 25+ Some College or More', 
                                        'Percent: 25+ Bachelor\'s Degree or More', 
                                        'Percent: 25+ Master\'s Degree or More',
                                        'Percent: 25+ Professional School Degree or More',
                                        'Percent: 25+ Doctorate Degree'])
    
    for i in range(len(branchDataset)):
        
        #Locating only column 'list of dict'
        libraryDictList = branchDataset.loc[i,'list of dict']
        branchName = branchDataset.loc[i,'BRANCH']
        
        #Removed set of brackets, just a dictionary now
        finalDictionary = libraryDictList[0]

        #Seperating into two columns
        dictToList = pd.DataFrame(finalDictionary.items(), columns= ['geoid','Percent Overlap'])
        libraryAndCensus = censusDemographics.merge(dictToList, on='geoid')

        weightedLibDemographics = branch_demographics(libraryAndCensus, branchName)
        
        dataByBranch = pd.concat([dataByBranch, weightedLibDemographics])
    dataByBranch = dataByBranch.reset_index()
    dataByBranch = dataByBranch.drop(columns = ['index'])
    return dataByBranch

In [22]:
#Calling the above function and renaming it to walking20Demographics
neighborhoodDemographics = allBranchDemographics(branchInfo)

Percent Population in Labor Force 16+: Employed
Percent Population in Labor Force 16+: Unemployed
Percent Households: Less Than $10,000
Percent Households: $10,000 to $14,999
Percent Households: $15,000 to $19,999
Percent Households: $20,000 to $24,999
Percent Households: $25,000 to $29,999
Percent Households: $30,000 to $34,999
Percent Households: $35,000 to $39,999
Percent Households: $40,000 to $44,999
Percent Households: $45,000 to $49,999
Percent Households: $50,000 to $59,999
Percent Households: $60,000 to $74,999
Percent Households: $75,000 to $99,999
Percent Households: $100,000 to $124,999
Percent Households: $125,000 to $149,999
Percent Households: $150,000 to $199,999
Percent Households: $200,000 or more
Percent Ages 18-64: Living in Poverty
Percent Ages 18-64: At or Above Poverty Level
Percent Households: With Public Assistance Income
Percent Households: No Public Assistance Income
Percent Owner and Renter Occupied Housing: No Vehicles
Percent Owner and Renter Occupied Hous

In [23]:
neighborhoodDemographics

,Branch,Percent of People with Internet Access,Percent of People without Internet Access,Percent of People with Computers,Percent of People without Computers,Percent of People with Computers and Internet,Percent Population in Labor Force 16+: Employed,Percent Population in Labor Force 16+: Unemployed,"Percent Households: Less Than $10,000","Percent Households: $10,000 to $14,999",...,Percent: 75 to 79 years,Percent: 80 to 84 years,Percent: 85 years and over,Percent: 25+ Less Than High School,Percent: 25+ High School or More,Percent: 25+ Some College or More,Percent: 25+ Bachelor's Degree or More,Percent: 25+ Master's Degree or More,Percent: 25+ Professional School Degree or More,Percent: 25+ Doctorate Degree
0,Albany Park,0.909940,0.090060,0.952462,0.047538,0.853041,0.945356,0.054644,0.057508,0.026106,...,0.020144,0.013852,0.016877,0.197002,0.802998,0.612690,0.426111,0.156380,0.039713,0.016772
1,Altgeld,0.870495,0.129505,0.926451,0.073549,0.846187,0.818177,0.181823,0.193472,0.052945,...,0.018016,0.012641,0.012953,0.110176,0.889824,0.570568,0.132852,0.042577,0.004715,0.001290
2,Archer Heights,0.803713,0.196287,0.834062,0.165938,0.770264,0.887683,0.112317,0.040106,0.039626,...,0.030270,0.012828,0.023844,0.257835,0.742165,0.360209,0.145143,0.040895,0.005648,0.002678
3,Austin,0.838028,0.161972,0.877664,0.121784,0.776312,0.904136,0.095864,0.118477,0.070640,...,0.027162,0.016595,0.007294,0.168420,0.831580,0.541913,0.269381,0.118897,0.030651,0.011131
4,Austin-Irving,0.852943,0.147057,0.921767,0.078233,0.822189,0.953150,0.046850,0.044079,0.027426,...,0.033432,0.015745,0.021586,0.104422,0.895578,0.600304,0.296879,0.101284,0.026887,0.014001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,West Pullman,0.832257,0.167743,0.889108,0.110892,0.807022,0.826266,0.173734,0.105075,0.049279,...,0.031183,0.024630,0.020161,0.118748,0.881252,0.590440,0.194724,0.083448,0.011345,0.004821
77,West Town,0.887932,0.046668,0.756990,0.033599,0.923999,0.965915,0.034085,0.032368,0.022077,...,0.008082,0.006530,0.003831,0.042601,0.957399,0.881096,0.750735,0.286695,0.092712,0.028974
78,"Whitney M. Young, Jr.",0.771061,0.228939,0.878638,0.121362,0.734837,0.839664,0.160336,0.143702,0.094549,...,0.020027,0.027906,0.026779,0.114952,0.885048,0.640804,0.273663,0.105575,0.026267,0.008458
79,Woodson Regional Library,0.855258,0.144742,0.901773,0.098227,0.812045,0.847638,0.152362,0.095327,0.054249,...,0.025783,0.028371,0.037407,0.109830,0.890170,0.620663,0.256082,0.100677,0.011119,0.006331


In [24]:
avgCycling

,Unnamed: 0,Branch,Average Cycling Time (Minutes)
0,0,Albany Park,4.437811
1,1,Altgeld,14.056106
2,2,Archer Heights,5.780903
3,3,Austin,7.555355
4,4,Austin-Irving,4.218488
...,...,...,...
76,76,West Pullman,6.569592
77,77,West Town,4.218355
78,78,"Whitney M. Young, Jr.",5.393937
79,79,Woodson Regional Library,6.357643


In [25]:
neighborhoodDemographics['Average Cycling Time (Min)'] = avgCycling['Average Cycling Time (Minutes)']

In [26]:
neighborhoodDemographics

,Branch,Percent of People with Internet Access,Percent of People without Internet Access,Percent of People with Computers,Percent of People without Computers,Percent of People with Computers and Internet,Percent Population in Labor Force 16+: Employed,Percent Population in Labor Force 16+: Unemployed,"Percent Households: Less Than $10,000","Percent Households: $10,000 to $14,999",...,Percent: 80 to 84 years,Percent: 85 years and over,Percent: 25+ Less Than High School,Percent: 25+ High School or More,Percent: 25+ Some College or More,Percent: 25+ Bachelor's Degree or More,Percent: 25+ Master's Degree or More,Percent: 25+ Professional School Degree or More,Percent: 25+ Doctorate Degree,Average Cycling Time (Min)
0,Albany Park,0.909940,0.090060,0.952462,0.047538,0.853041,0.945356,0.054644,0.057508,0.026106,...,0.013852,0.016877,0.197002,0.802998,0.612690,0.426111,0.156380,0.039713,0.016772,4.437811
1,Altgeld,0.870495,0.129505,0.926451,0.073549,0.846187,0.818177,0.181823,0.193472,0.052945,...,0.012641,0.012953,0.110176,0.889824,0.570568,0.132852,0.042577,0.004715,0.001290,14.056106
2,Archer Heights,0.803713,0.196287,0.834062,0.165938,0.770264,0.887683,0.112317,0.040106,0.039626,...,0.012828,0.023844,0.257835,0.742165,0.360209,0.145143,0.040895,0.005648,0.002678,5.780903
3,Austin,0.838028,0.161972,0.877664,0.121784,0.776312,0.904136,0.095864,0.118477,0.070640,...,0.016595,0.007294,0.168420,0.831580,0.541913,0.269381,0.118897,0.030651,0.011131,7.555355
4,Austin-Irving,0.852943,0.147057,0.921767,0.078233,0.822189,0.953150,0.046850,0.044079,0.027426,...,0.015745,0.021586,0.104422,0.895578,0.600304,0.296879,0.101284,0.026887,0.014001,4.218488
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,West Pullman,0.832257,0.167743,0.889108,0.110892,0.807022,0.826266,0.173734,0.105075,0.049279,...,0.024630,0.020161,0.118748,0.881252,0.590440,0.194724,0.083448,0.011345,0.004821,6.569592
77,West Town,0.887932,0.046668,0.756990,0.033599,0.923999,0.965915,0.034085,0.032368,0.022077,...,0.006530,0.003831,0.042601,0.957399,0.881096,0.750735,0.286695,0.092712,0.028974,4.218355
78,"Whitney M. Young, Jr.",0.771061,0.228939,0.878638,0.121362,0.734837,0.839664,0.160336,0.143702,0.094549,...,0.027906,0.026779,0.114952,0.885048,0.640804,0.273663,0.105575,0.026267,0.008458,5.393937
79,Woodson Regional Library,0.855258,0.144742,0.901773,0.098227,0.812045,0.847638,0.152362,0.095327,0.054249,...,0.028371,0.037407,0.109830,0.890170,0.620663,0.256082,0.100677,0.011119,0.006331,6.357643


In [27]:
neighborhoodDemographics.to_csv('../library neighborhoods/avgCyclingTimeNeighborhoodsDems.csv')